![Ironhack logo](https://i.imgur.com/1QgrNNw.png)

<body>
    <p style="font-size:28px;text-align:center"><b>Project 03 | Web Scrapping & API</b></p>
</body>

# Introduction

The objective of the first part of this project was collect data to answer a problem statement, practicing web scrapping and using API.

---

<body>
    <p style="font-size:20px"><b>Problem Statement</b></p>
</body>

_What some the TikTok viral songs have in common?_

---

To answer this problem, 69 songs, which were obtained from the **PopSugar** website, were analyzed. The post that contained this list of songs was made in March 27th, 2020 by Hedy Phillips around the same time people started to quarantine, because of the COVIVD-19 pandemic, and people began to use it more to spend their time at home.

The sources of information used to gather data were **Spotify**,  **Last.fm** and **MusicBrainz**.

---

Sources:
- Websites:
  - PopSugar: https://www.popsugar.com/entertainment/popular-tiktok-songs-47289804?stream_view=1#photo-47289832
  - MusicBrainz:https://musicbrainz.org/genres
 
- APIs
  - Spotify API: https://developer.spotify.com/
  - Spotipy (Spotify API wrapper for Python): https://spotipy.readthedocs.io/en/2.15.0/
  - Last.fm API: https://www.last.fm/api


# Setup

## Import

In [1]:
import os
import re
import requests
from time import sleep

import numpy as np
import pandas as pd
import spotipy

from bs4 import BeautifulSoup
from dotenv import load_dotenv, find_dotenv
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from tqdm.auto import tqdm

# Web Scrapping

The web scrapping was necessary to collect the following data:

<table>
  <thead>
    <tr>
      <th>INFORMATION</th>
      <th>SOURCE</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>69 TikTok viral songs</td>
      <td>PopSugar</td>
    </tr>
    <tr>
      <td>Music genres</td>
      <td>MusicBrainz</td>
    </tr>
  </tbody>
</table>



## List of viral songs on TikTok

### Get response

In [2]:
# Get response from the url and check it
url = 'https://www.popsugar.com/entertainment/popular-tiktok-songs-47289804?stream_view=1#photo-47289832'
response = requests.get(url)
response

<Response [200]>

### Data Collection

In [3]:
# Get the content in the url
content_popsugar = BeautifulSoup(response.text)

# Get the date the post was made
popsugar_date = content_popsugar.find('time').text.replace('\n', '').strip()

# Get only the songs and artists
popsugar_html = content_popsugar.find_all('span', attrs={'class': 'count-copy'})

In [4]:
# Conver the list 'html_popsugar' to a Pandas DataFrame
df_base = pd.DataFrame([re.split(' by ', song.text.replace('"', '').strip()) for song in popsugar_html], 
                       columns=['song', 'artists'])

# Check the result
df_base

,song,artists
0,Roxanne,Arizona Zervas
1,Say So,Doja Cat
2,My Oh My,Camila Cabello feat. DaBaby
3,Moon,Kid Francescoli
4,Vibe,Cookiee Kawaii
...,...,...
64,What the Hell,Avril Lavigne
65,Towards the Sun,Rihanna
66,I Think I'm OKAY,"Machine Gun Kelly, YUNGBLUD, and Travis Barker"
67,Myself,Bazzi


### Data Cleaning

In [5]:
# Create a column with a list of artists for each song
df_base['artists_list'] = [re.split(',* and |, * | [Ff]eat. ', artists.strip()) for artists in df_base.artists]

# Create a column with the number of artists for each song
df_base['number_artists'] = df_base.artists_list.apply(len)

In [6]:
# Check possible number of artists for one song
df_base.number_artists.value_counts()

1    48
2    18
3     3
Name: number_artists, dtype: int64

Seeing the result above, the maximum number of artists for a song is 3.

In [7]:
# Create columns to stores one artist each
df_base['artist_1'] = df_base.artists_list.apply(lambda x : x[0] if len(x) >= 1 else 'no-artist')
df_base['artist_2'] = df_base.artists_list.apply(lambda x : x[1] if len(x) >= 2 else 'no-artist')
df_base['artist_3'] = df_base.artists_list.apply(lambda x : x[2] if len(x) == 3 else 'no-artist')

In [8]:
df_base.head()

,song,artists,artists_list,number_artists,artist_1,artist_2,artist_3
0,Roxanne,Arizona Zervas,[Arizona Zervas],1,Arizona Zervas,no-artist,no-artist
1,Say So,Doja Cat,[Doja Cat],1,Doja Cat,no-artist,no-artist
2,My Oh My,Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,Camila Cabello,DaBaby,no-artist
3,Moon,Kid Francescoli,[Kid Francescoli],1,Kid Francescoli,no-artist,no-artist
4,Vibe,Cookiee Kawaii,[Cookiee Kawaii],1,Cookiee Kawaii,no-artist,no-artist


### Make backup dataframe

In [9]:
df_base_raw_bck = df_base.copy()

## List of music genre

### Get the response

In [10]:
# Get response from the url and check it
url = 'https://musicbrainz.org/genres'
response = requests.get(url)
response

<Response [200]>

### Data collection

In [11]:
# Get the content in the url
musicbrainz_content = BeautifulSoup(response.content)

# Create a list with the music genres listed in the url
musicbrainz_genre = [genre.text for genre in musicbrainz_content.find_all('bdi')]

The data cleaning for this list will be made later.

# Spotify

From the Spotify and with the Spotipy's help, some data about each song will be gathered. It is relevant to point that there is a possibility that some songs may not be in the Spotify's library.

## Connecting to the API

In [41]:
load_dotenv(find_dotenv())

True

In [42]:
cid = os.getenv('spotify_p03_key')
csecret = os.getenv('spotify_p03_secret')
cc_manager = SpotifyClientCredentials(client_id=cid, client_secret=csecret)
sp = spotipy.Spotify(client_credentials_manager=cc_manager)

## Songs

### Search information about each song

In [14]:
# Search information about each song, using the Spotipy
spotify_songs = [sp.search(q=df_base.iloc[index, 0], type='track', limit=50) for index in tqdm(df_base.index)]

In [15]:
# Check if there are 69 items in this list
len(spotify_songs)

69

In [16]:
# Add a column in the dataframe with the data that were just collected
df_base['spotify_search'] = spotify_songs

In [17]:
# Check the result
df_base.head()

,song,artists,artists_list,number_artists,artist_1,artist_2,artist_3,spotify_search
0,Roxanne,Arizona Zervas,[Arizona Zervas],1,Arizona Zervas,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...
1,Say So,Doja Cat,[Doja Cat],1,Doja Cat,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...
2,My Oh My,Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,Camila Cabello,DaBaby,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...
3,Moon,Kid Francescoli,[Kid Francescoli],1,Kid Francescoli,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...
4,Vibe,Cookiee Kawaii,[Cookiee Kawaii],1,Cookiee Kawaii,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...


In [28]:
# Function to add new information in a copy of the dataframe
def get_spotify_track_info(df):
    
    '''
    Filters some data of the songs and adds them to a copy of the dataframe
    
    Args:
    -----
        df (Pandas DataFrame): a dataframe containing the songs and their artists
    
    Returns:
    --------
        df_copy (Pandas DataFrame): a copy of the dataframe with some new information appended
    '''
    
    # Create auxiliary empty lists (final lists)
    list_spotify_track_id = []
    list_spotify_track_duration = []
    list_spotify_track_popularity = []
    list_spotify_album_release_date = []
    list_spotify_track_explicit = []
    lists_spotify = [list_spotify_track_id, list_spotify_track_duration, list_spotify_track_popularity,
                     list_spotify_album_release_date, list_spotify_track_explicit]
    
    
    # Check for each row of the dataframe
    for index in df.index:
        
        # Information necessary from the dataframe to use during the process
        song_name = df.iloc[index, 0].lower()
        artists_list = [artist.lower() for artist in df.iloc[index, 2]]
        total_artists = df.iloc[index, 3]
        mask = df.iloc[index, 7]['tracks']['items']
        
        # If the track was not found in the Spotify library, a 'not-found' string is added to the final lists
        if len(mask) == 0:
            for lst in lists_spotify:
                lst.append('not-found')
            #print(f'{index} - {song_name} - NOT FOUND')
        
        # If the track was found in the Spotify
        else:
            
            # Variable necessary to check if the information about a song has been added to the final lists
            added = 0
            
            # For each track it was listed 50 tracks related to the query 
            for idx, each_found in enumerate(mask):
                
                # Information necessary from the Spotify API to use during the process
                track_name = mask[idx]['name'].lower()
                track_id = mask[idx]['id']
                track_duration = mask[idx]['duration_ms']
                track_popularity = mask[idx]['popularity']
                album_release_date = mask[idx]['album']['release_date']
                track_explicit = mask[idx]['explicit']
                n_artists = len(mask[idx]['artists'])
                first_artist_name = mask[idx]['artists'][0]['name'].lower()
            
                # Check if the name of the song, the artists from both sources match and if an information about the
                # song has been added to the final lists 
                if ((song_name in track_name) & (total_artists == n_artists) & (first_artist_name in artists_list)
                    & (added == 0)):
                    list_spotify_track_id.append(track_id)
                    list_spotify_track_duration.append(track_duration)
                    list_spotify_track_popularity.append(track_popularity)
                    list_spotify_album_release_date.append(album_release_date)
                    list_spotify_track_explicit.append(track_explicit)
                    added += 1
                    #print(f'{index} - {track_name} - {track_id}')
                        
                
                # If the track found in the search is not a math, itis the last one and information about the track 
                # has not been added to the final list, then add a 'not-found' string to the final lists
                elif (idx == len(mask) - 1) & (added == 0):
                    for lst in lists_spotify:
                        lst.append('not-found')
                    #print(f'{index} - {song_name} - NOT FOUND')
    
    # Make a copy of the dataframe
    df_copy = df.copy()
    
    # Add columns with the desired information
    # Not an inplace process
    df_copy['sp_id'] = list_spotify_track_id
    df_copy['sp_duration_ms'] = list_spotify_track_duration
    df_copy['sp_popularity'] = list_spotify_track_popularity
    df_copy['sp_release_date'] = list_spotify_album_release_date
    df_copy['sp_explicit'] = list_spotify_track_explicit
                    
    return df_copy

In [29]:
# Add desired information to the dataframe
df_base = get_spotify_track_info(df_base)

# Check the result
df_base.head()

,song,artists,artists_list,number_artists,artist_1,artist_2,artist_3,spotify_search,sp_id,sp_duration_ms,sp_popularity,sp_release_date,sp_explicit
0,Roxanne,Arizona Zervas,[Arizona Zervas],1,Arizona Zervas,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,696DnlkuDOXcMAnKlTgXXK,163636,89,2019-10-10,True
1,Say So,Doja Cat,[Doja Cat],1,Doja Cat,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,3Dv1eDb0MEgF93GpLXlucZ,237893,89,2019-11-07,True
2,My Oh My,Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,Camila Cabello,DaBaby,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,3yOlyBJuViE2YSGn3nVE1K,170746,83,2019-12-06,False
3,Moon,Kid Francescoli,[Kid Francescoli],1,Kid Francescoli,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,24upABZ8A0sAepfu91sEYr,390638,70,2017-03-03,False
4,Vibe,Cookiee Kawaii,[Cookiee Kawaii],1,Cookiee Kawaii,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,4gOgQTv9RYYFZ1uQNnlk3q,83940,72,2019-03-29,True


### Find the audio features for each song

In [22]:
# Search in the API wrapper
spotify_audio_features = [sp.audio_features(track_id)  if track_id != 'not-found'else 'not-found' 
                          for track_id in tqdm(df_base.sp_id)]

In [23]:
# Check if there are 69 items in this list
len(spotify_audio_features)

69

In [24]:
# Function to add new information in a copy of the dataframe
def get_spotify_audio_features(df, audio_features: list):
    
    '''
    Adds new information from a list to a copy of the dataframe
    
    Args:
    -----
        df (Pandas DataFrame): a dataframe containing the songs and their artists
    
    Returns:
    --------
        df_copy (Pandas DataFrame): a copy of the dataframe with some new information appended
    '''
    
    # Create auxiliary empty lists (final lists)
    list_danceability = []
    list_energy = []
    list_key = []
    list_loudness = []
    list_mode = []
    list_speechiness = []
    list_acousticness = []
    list_instrumentalness = []
    list_liveness = []
    list_valence = []
    list_tempo = []
    list_time_signature = []
    lists_features = [list_danceability, list_energy, list_key, list_loudness, list_mode, list_speechiness, 
                      list_acousticness, list_instrumentalness, list_liveness, list_valence, list_tempo,
                      list_time_signature]
    
    # Check for each row of the dataframe
    for index in df.index:
        
        # Get the track's Spotify id
        track_id = df.iloc[index, 8]
        
        # If the track was not found in the Spotify library, a 'not-found' string is added to the final lists
        if track_id == 'not-found':
            
            for lst in lists_features:
                lst.append('not-found')
    
        # If the track was found in the Spotify library
        else:
            
            # Add the information to the final lists
            list_danceability.append(audio_features[index][0]['danceability'])
            list_energy.append(audio_features[index][0]['energy'])
            list_key.append(audio_features[index][0]['key'])
            list_loudness.append(audio_features[index][0]['loudness'])
            list_mode.append(audio_features[index][0]['mode'])
            list_speechiness.append(audio_features[index][0]['speechiness'])
            list_acousticness.append(audio_features[index][0]['acousticness'])
            list_instrumentalness.append(audio_features[index][0]['instrumentalness'])
            list_liveness.append(audio_features[index][0]['liveness'])
            list_valence.append(audio_features[index][0]['valence'])
            list_tempo.append(audio_features[index][0]['tempo'])
            list_time_signature.append(audio_features[index][0]['time_signature'])
     
     # Make a copy of the dataframe
    df_copy = df.copy()
    
    # Add columns with the desired information
    # Not an inplace process
    df_copy['sp_danceability'] = list_danceability
    df_copy['sp_energy'] = list_energy
    df_copy['sp_key'] = list_key
    df_copy['sp_loudness'] = list_loudness
    df_copy['sp_mode'] = list_mode
    df_copy['sp_speechiness'] = list_speechiness
    df_copy['sp_acousticness'] = list_acousticness
    df_copy['sp_instrumentalness'] = list_instrumentalness
    df_copy['sp_liveness'] = list_liveness
    df_copy['sp_valence'] = list_valence
    df_copy['sp_tempo'] = list_tempo
    df_copy['sp_time_signature'] = list_time_signature
    
    return df_copy

In [26]:
df_base = get_spotify_audio_features(df_base, spotify_audio_features)
df_base.head()

,song,artists,artists_list,number_artists,artist_1,artist_2,artist_3,spotify_search,sp_id,sp_duration_ms,...,sp_key,sp_loudness,sp_mode,sp_speechiness,sp_acousticness,sp_instrumentalness,sp_liveness,sp_valence,sp_tempo,sp_time_signature
0,Roxanne,Arizona Zervas,[Arizona Zervas],1,Arizona Zervas,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,696DnlkuDOXcMAnKlTgXXK,163636,...,6,-5.616,0,0.148,0.0522,0,0.46,0.457,116.735,5
1,Say So,Doja Cat,[Doja Cat],1,Doja Cat,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,3Dv1eDb0MEgF93GpLXlucZ,237893,...,11,-4.577,0,0.158,0.256,3.57e-06,0.0904,0.786,110.962,4
2,My Oh My,Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,Camila Cabello,DaBaby,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,3yOlyBJuViE2YSGn3nVE1K,170746,...,8,-6.024,1,0.0296,0.018,1.29e-05,0.0887,0.383,105.046,4
3,Moon,Kid Francescoli,[Kid Francescoli],1,Kid Francescoli,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,24upABZ8A0sAepfu91sEYr,390638,...,7,-10.002,1,0.0345,0.288,0.856,0.102,0.0584,117.986,4
4,Vibe,Cookiee Kawaii,[Cookiee Kawaii],1,Cookiee Kawaii,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,4gOgQTv9RYYFZ1uQNnlk3q,83940,...,10,-8.719,1,0.344,0.0635,0.00932,0.118,0.175,159.947,4


## Artists

### Get information about each artist

In [81]:
# Check unique artists

list_unique_artists = []

for index in df_base.index:
    if df_base.iloc[index, 8] != 'not-fond':
        for each_list in df_base.iloc[index, 2]:   
            list_unique_artists.append(each_list)
            
    
len(set(list_unique_artists))

91

In [33]:
# Function to add new information in a copy of the dataframe
def get_spotify_artists_id(df):
    
    '''
    Creates a new dataframe about the artists.
    
    Args:
    -----
        df (Pandas DataFrame): a dataframe containing the songs and their artists
    
    Returns:
    --------
        df_artists_id (Pandas DataFrame): a new dataframe with the name and Spotify ID of the artists
    '''
    
    # Create auxiliary empty dictionary
    dict_artists_ids = {}
      
    # Check for each row of the dataframe
    for index in df.index:
                      
        # Information necessary from the dataframe to use during the process
        song_name = df.iloc[index, 0].lower()
        artists_list = [artist.lower() for artist in df.iloc[index, 2]]
        total_artists = df.iloc[index, 3]
        mask = df.iloc[index, 7]['tracks']['items']
        added = 0
        
        # For each track it was listed 50 tracks related to the query 
        for idx, each_found in enumerate(mask):
                
            # Information necessary from the Spotify API to use during the process
            track_name = mask[idx]['name'].lower()
            n_artists = len(mask[idx]['artists'])
            first_artist_name = mask[idx]['artists'][0]['name'].lower()
            
            # Check if the name of the song, the artists from both sources match and if an information about the
            # song has been added to the final lists 
            if ((song_name in track_name) & (total_artists == n_artists) & (first_artist_name in artists_list)
                & (added == 0)):
                
                for artist in mask[idx]['artists']:
                    artist_name = artist['name']
                    artist_id = artist['id']
                        
                    # Add to dict
                    dict_artists_ids[artist_name] = artist_id
                    
    # Create a Pandas DataFrame
    
    df_artists_id = pd.DataFrame(dict_artists_ids.items(), columns=['artist', 'sp_id'])
                    
    return df_artists_id

In [36]:
# Create a Pandas DataFrame with artists
df_artists = get_spotify_artists_id(df_base)

# Check result
df_artists.head()

,artist,sp_id
0,Arizona Zervas,0vRvGUQVUjytro0xpb26bs
1,Doja Cat,5cj0lLjcoR7YOSnhnX0Po5
2,Camila Cabello,4nDoRrQiYLoBzwC5BhVJzF
3,DaBaby,4r63FhuTkUYltbVAg5TQnk
4,Kid Francescoli,2G7QgTep5IsJHGHm1hXygD


In [44]:
# Search in the API wrapper
spotify_artists_info = [sp.artist(artist) for artist in  tqdm(df_artists.sp_id)]

In [60]:
spotify_artists_info

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/0vRvGUQVUjytro0xpb26bs'},
  'followers': {'href': None, 'total': 484894},
  'genres': ['pop rap', 'rap', 'rhode island rap'],
  'href': 'https://api.spotify.com/v1/artists/0vRvGUQVUjytro0xpb26bs',
  'id': '0vRvGUQVUjytro0xpb26bs',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/d549aadbb8b3a254fdc8e5ac93535a706463dce6',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/3e8de8dd7f9380757f655028ef3209e73a688a00',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/b034f5b9aea4d39eef47278e8f8702e116be66cb',
    'width': 160}],
  'name': 'Arizona Zervas',
  'popularity': 80,
  'type': 'artist',
  'uri': 'spotify:artist:0vRvGUQVUjytro0xpb26bs'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/5cj0lLjcoR7YOSnhnX0Po5'},
  'followers': {'href': None, 'total': 3414060},
  'genres': ['la indie', 'pop'],
  'href': 'https://api.spotify.com/v1/artists/5cj0lL

In [49]:
# Add a column in the dataframe with the data that were just collected
df_artists['spotify_artist'] = spotify_artists_info

# Check the result
df_artists.head()

,artist,sp_id,spotify_artist
0,Arizona Zervas,0vRvGUQVUjytro0xpb26bs,{'external_urls': {'spotify': 'https://open.sp...
1,Doja Cat,5cj0lLjcoR7YOSnhnX0Po5,{'external_urls': {'spotify': 'https://open.sp...
2,Camila Cabello,4nDoRrQiYLoBzwC5BhVJzF,{'external_urls': {'spotify': 'https://open.sp...
3,DaBaby,4r63FhuTkUYltbVAg5TQnk,{'external_urls': {'spotify': 'https://open.sp...
4,Kid Francescoli,2G7QgTep5IsJHGHm1hXygD,{'external_urls': {'spotify': 'https://open.sp...


In [63]:
df_artists.iloc[0, 2]['id']

'0vRvGUQVUjytro0xpb26bs'

In [71]:
# Function to add new information in a copy of the dataframe
def get_spotify_artist_info(df):
    
    '''
    Adds information about the artists to a copy of the dataframe.
    
    Args:
    -----
        df (Pandas DataFrame): a dataframe containing the artists and their Spotify ID
    
    Returns:
    --------
        df_copy (Pandas DataFrame): a copy of the dataframe with some new information appended
    '''
    
    # Create auxiliary empty lists (final lists)
    list_spotify_artist_genres = []
    list_spotify_artist_popularity = []
    list_spotify_artist_followers = []
    
    # Check for each row of the dataframe
    for index in df.index:
        
        sp_id = df.iloc[index, 1]
        mask = df.iloc[index, 2]
        search_sp_id = mask['id']
        #artist_name = mask['name']
        
        # Check if 'id's match
        if sp_id == search_sp_id:
            #print(f'{index} - {artist_name}: OK - {n_artist}')
            
            search_sp_genres = mask['genres']
            list_spotify_artist_genres.append(search_sp_genres)
            
            search_sp_popularity = mask['popularity']
            list_spotify_artist_popularity.append(search_sp_popularity)
            
            search_sp_followers = mask['followers']['total']
            list_spotify_artist_followers.append(search_sp_followers)
    
    # Make a copy of the dataframe
    df_copy = df.copy()
    
    # Add columns with the desired information
    # Not an inplace process
    df_copy['sp_genres'] = list_spotify_artist_genres
    df_copy['sp_popularity'] = list_spotify_artist_popularity
    df_copy['sp_followers'] = list_spotify_artist_followers
                    
    return df_copy

In [73]:
# Add desired information to the dataframe
df_artists = get_spotify_artist_info(df_artists)

# Check the result
df_artists.head()

,artist,sp_id,spotify_artist,sp_genres,sp_popularity,sp_followers
0,Arizona Zervas,0vRvGUQVUjytro0xpb26bs,{'external_urls': {'spotify': 'https://open.sp...,"[pop rap, rap, rhode island rap]",80,484894
1,Doja Cat,5cj0lLjcoR7YOSnhnX0Po5,{'external_urls': {'spotify': 'https://open.sp...,"[la indie, pop]",88,3414060
2,Camila Cabello,4nDoRrQiYLoBzwC5BhVJzF,{'external_urls': {'spotify': 'https://open.sp...,"[dance pop, pop, post-teen pop]",87,17956655
3,DaBaby,4r63FhuTkUYltbVAg5TQnk,{'external_urls': {'spotify': 'https://open.sp...,"[north carolina hip hop, rap]",95,4299650
4,Kid Francescoli,2G7QgTep5IsJHGHm1hXygD,{'external_urls': {'spotify': 'https://open.sp...,"[electronica, french indie pop, french indietr...",61,82042


## Playlists

In [86]:
# Search in the API wrapper
spotify_tiktok = sp.search(q='tiktok', type='playlist', limit=50)

In [87]:
len(spotify_tiktok['playlists']['items'])

50

# Last.fm

## Connecting to the API and collecting data

Example of how to connect to the API and collect information about a song

```python
response = lastfm_get({
    'method': 'track.getInfo',
    'track': 'My Oh My',
    'artist': 'Camila Cabello'
})```

In [27]:
lastfm_key = os.getenv('lastfm_p03_key')
lastfm_user = 'gnakasato'

In [28]:
# Function to connect to the API and get data
def lastfm_get(payload):
    
    '''
    Connects to the Last.fm API
    
    Args:
    -----
        payload (dict): a dictionary containing additional information to connect to the Last.fm API
    
    Returns:
    --------
        response: the response for the API connection request
    '''
    
    # Define the headers and the url
    headers = {'user-agent': lastfm_user}
    url = 'http://ws.audioscrobbler.com/2.0/'
    
    # Add API key and format to the payload
    payload['api_key'] = lastfm_key
    payload['format=json'] = lastfm_key
    
    response = requests.get(url, headers=headers, params=payload)
    
    return response

In [29]:
# Create an auxiliary list
lastfm_track_info = []

# Connect to the API and collect data
for index in tqdm(range(df_base.shape[0])):
    
    # Connect to the API
    response = lastfm_get({'method': 'track.getInfo',
                           'track': df_base.iloc[index, 0],
                           'artist': df_base.iloc[index, 4],
                           'autocorrect[0|1]': '1'})
    
    # Convert the data into a more amicable format
    track_info = BeautifulSoup(response.text)
    
    # Add the data to the auxiliary list
    lastfm_track_info.append(track_info)
    
    # Wait 1 second to check the next track
    sleep(1)

## Data Cleaning

In [30]:
# Create a list with the tags of each track
# If the track was not found in the API, the value to the corresponding track is 'not-found'
lastfm_tags = [track.find_all('tag') if track.text.replace('\n\n', '') != 'Track not found' else 'not-found' for track 
                in lastfm_track_info]

# Check the result for the first two tracks
lastfm_tags[:2]

[[<tag><name>2019</name>
  <url>https://www.last.fm/tag/2019</url>
  </tag>,
  <tag><name>2010s</name>
  <url>https://www.last.fm/tag/2010s</url>
  </tag>,
  <tag><name>arizona zervas</name>
  <url>https://www.last.fm/tag/arizona+zervas</url>
  </tag>,
  <tag><name>Hip-Hop</name>
  <url>https://www.last.fm/tag/Hip-Hop</url>
  </tag>,
  <tag><name>rap</name>
  <url>https://www.last.fm/tag/rap</url>
  </tag>],
 [<tag><name>pop</name>
  <url>https://www.last.fm/tag/pop</url>
  </tag>,
  <tag><name>Disco</name>
  <url>https://www.last.fm/tag/Disco</url>
  </tag>,
  <tag><name>rap</name>
  <url>https://www.last.fm/tag/rap</url>
  </tag>,
  <tag><name>Hip-Hop</name>
  <url>https://www.last.fm/tag/Hip-Hop</url>
  </tag>,
  <tag><name>female vocalists</name>
  <url>https://www.last.fm/tag/female+vocalists</url>
  </tag>]]

The idea is to create a nested list with lists of tags of each track

In [31]:
# Create auxiliary empty lists 
tracks_tags = []  # Final list with the tags
tags_exist = []  # List with all tags

# Get the tag names for each track from the raw (messy) data
for track_tag in lastfm_tags:
    
    # If the track was not found in the Last.fm API
    if track_tag == 'not-found':
        tracks_tags.append(['not-found'])
    
    # If the track was found, but there is no tag related to the track
    elif len(track_tag) == 0:
        tracks_tags.append(['no-tag'])
    
    # If the track was found and there are tags related to the track
    else:
        
        # Create an auxiliary list 
        # Before checking each track, this list needs to be cleared, so it stores a list of messy tags for each track
        each_tag_lists = []
        
        # Each track has a list with messy tags, so it is necessary to clean this data, checking each tag for each track
        for each_tag in track_tag:
            
            # Get only the tag name, but this process creates a list with the tag name
            tag_list = each_tag.find('name')
            
            # Add each tag (messy data) in an auxiliary list
            each_tag_lists.append(tag_list)
            
            # Create an auxiliary list
            # Before checking each tag of a track, this list needs to be cleared, so it stores a list of clean data of
            # tags for each track
            each_track_tags = []
            
            # Each tag name of a track is inside a list
            for each_one in each_tag_lists:
                
                # Clean the data for each tag name
                tag = each_one.text.lower().replace('-', ' ')
                
                # Add the tag (clean data) with all lower case letters in an auxiliary list if the tag is one of the
                # genres listed in the 'musicbrainz_genre'
                if tag in musicbrainz_genre:
                    each_track_tags.append(tag)
                    tags_exist.append(tag)
                
        # Add each list of tags (clean) for one track in a final list
        tracks_tags.append(each_track_tags)
        #print(f'\n{tracks_tags}\n')
        
# Add a column in the 'popsugar_df' with the tags found in Last.fm API
df_base['lastfm_tags'] = tracks_tags

In [32]:
# Check the result
df_base.head()

,song,artists,artists_list,number_artists,artist_1,artist_2,artist_3,spotify_search,sp_id,sp_duration_ms,...,sp_loudness,sp_mode,sp_speechiness,sp_acousticness,sp_instrumentalness,sp_liveness,sp_valence,sp_tempo,sp_time_signature,lastfm_tags
0,Roxanne,Arizona Zervas,[Arizona Zervas],1,Arizona Zervas,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,696DnlkuDOXcMAnKlTgXXK,163636,...,-5.616,0,0.148,0.0522,0,0.46,0.457,116.735,5,[hip hop]
1,Say So,Doja Cat,[Doja Cat],1,Doja Cat,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,3Dv1eDb0MEgF93GpLXlucZ,237893,...,-4.577,0,0.158,0.256,3.57e-06,0.0904,0.786,110.962,4,"[pop, disco, hip hop]"
2,My Oh My,Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,Camila Cabello,DaBaby,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,3yOlyBJuViE2YSGn3nVE1K,170746,...,-6.024,1,0.0296,0.018,1.29e-05,0.0887,0.383,105.046,4,[pop]
3,Moon,Kid Francescoli,[Kid Francescoli],1,Kid Francescoli,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,24upABZ8A0sAepfu91sEYr,390638,...,-10.002,1,0.0345,0.288,0.856,0.102,0.0584,117.986,4,"[chillout, indie pop]"
4,Vibe,Cookiee Kawaii,[Cookiee Kawaii],1,Cookiee Kawaii,no-artist,no-artist,{'tracks': {'href': 'https://api.spotify.com/v...,4gOgQTv9RYYFZ1uQNnlk3q,83940,...,-8.719,1,0.344,0.0635,0.00932,0.118,0.175,159.947,4,[no-tag]


# Final Dataframe

In [103]:
df = df_base.drop(['spotify_search'], axis=1)
df.head()

,song,artists,artists_list,number_artists,artist_1,artist_2,artist_3,sp_id,sp_duration_ms,sp_popularity,...,sp_loudness,sp_mode,sp_speechiness,sp_acousticness,sp_instrumentalness,sp_liveness,sp_valence,sp_tempo,sp_time_signature,lastfm_tags
0,Roxanne,Arizona Zervas,[Arizona Zervas],1,Arizona Zervas,no-artist,no-artist,696DnlkuDOXcMAnKlTgXXK,163636,89,...,-5.616,0,0.148,0.0522,0,0.46,0.457,116.735,5,[hip hop]
1,Say So,Doja Cat,[Doja Cat],1,Doja Cat,no-artist,no-artist,3Dv1eDb0MEgF93GpLXlucZ,237893,89,...,-4.577,0,0.158,0.256,3.57e-06,0.0904,0.786,110.962,4,"[pop, disco, hip hop]"
2,My Oh My,Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,Camila Cabello,DaBaby,no-artist,3yOlyBJuViE2YSGn3nVE1K,170746,83,...,-6.024,1,0.0296,0.018,1.29e-05,0.0887,0.383,105.046,4,[pop]
3,Moon,Kid Francescoli,[Kid Francescoli],1,Kid Francescoli,no-artist,no-artist,24upABZ8A0sAepfu91sEYr,390638,70,...,-10.002,1,0.0345,0.288,0.856,0.102,0.0584,117.986,4,"[chillout, indie pop]"
4,Vibe,Cookiee Kawaii,[Cookiee Kawaii],1,Cookiee Kawaii,no-artist,no-artist,4gOgQTv9RYYFZ1uQNnlk3q,83940,73,...,-8.719,1,0.344,0.0635,0.00932,0.118,0.175,159.947,4,[no-tag]
